<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-Image-Data" data-toc-modified-id="Loading-Image-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading Image Data</a></span><ul class="toc-item"><li><span><a href="#Initial-Examination" data-toc-modified-id="Initial-Examination-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initial Examination</a></span><ul class="toc-item"><li><span><a href="#Exercise:-Make-a-plot-of-a-slice-of-the-brain-(bonus:-and-the-corresponding-segmentation)." data-toc-modified-id="Exercise:-Make-a-plot-of-a-slice-of-the-brain-(bonus:-and-the-corresponding-segmentation).-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span><em>Exercise</em>: Make a plot of a slice of the brain (bonus: and the corresponding segmentation).</a></span></li></ul></li></ul></li><li><span><a href="#xArray-Dataset" data-toc-modified-id="xArray-Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>xArray Dataset</a></span></li><li><span><a href="#Advanced-Visualization" data-toc-modified-id="Advanced-Visualization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Advanced Visualization</a></span><ul class="toc-item"><li><span><a href="#Simple-Slice-Viewing-with-Holoviews" data-toc-modified-id="Simple-Slice-Viewing-with-Holoviews-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simple Slice Viewing with Holoviews</a></span><ul class="toc-item"><li><span><a href="#Exercise:-Why-do-we-have-to-flip-the-y-axis?-How-can-we-do-this-within-the-data,-before-plotting?" data-toc-modified-id="Exercise:-Why-do-we-have-to-flip-the-y-axis?-How-can-we-do-this-within-the-data,-before-plotting?-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span><em>Exercise</em>: Why do we have to flip the y axis? How can we do this within the data, before plotting?</a></span></li></ul></li><li><span><a href="#Multi-slice-Viewing" data-toc-modified-id="Multi-slice-Viewing-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Multi-slice Viewing</a></span></li><li><span><a href="#Alpha-Blending" data-toc-modified-id="Alpha-Blending-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Alpha Blending</a></span></li></ul></li></ul></div>

# Medical Imaging Analysis and xArray
Computer vision projects are commonplace in data science.  Typically, we have a large set of input images that need some sort of preprocessing and manipulation before we can crank them through some sort of ML architecture.  Ideally, we would like to have all images in a data structure in which we can quickly and easily analyze their content, perform operations, group and index, assign metadata and additional features, and so on.  Also, it would be nice if we could use a similar structure and API to `pandas` in order to minimize aggravation on our end.

[xArray](http://xarray.pydata.org/en/stable/) is a data structure built with multi-dimensional data in mind.  It is modeled very closely after `pandas`, and can be used to store a large set of images and additional data.

The dataset we'll be examining is a series of abdominal MRI sequences.

## Loading Image Data
Medical image data is typically stored as one of a number of image formats (DICOM, Nifti, MGH, etc), in order to maintain necessary metadata that corresponds to (among other things) the physical origin and spacing between the pixels of an image.  This information is not including in a simple array, which is needed when eventually invoking a tool like `pytorch`.  Because of this, we'll need to use a DICOM/Nifti image reader, so we'll be using `SimpleITK` to help with some image manipulation.

In [ ]:
# Autoload if needed
%matplotlib inline

In [ ]:
# Plotting setup
import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['figure.figsize'] = (10,8)
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

# Basic imports
import sys, os
import numpy as np
import pandas as pd
import xarray as xr
import SimpleITK as sitk
import nibabel as nib
import pickle as pkl
from pathlib import Path



### Initial Examination
We're going to look at brain images for this module.  Specifically, they are 3D volumes with associated segmentation maps.  We'd like to view them and manipulate them in a manner that's fast and easy, and integrates easily into our workflow and data processing pipeline.

In [ ]:
ls /pghbio/dbmi/batmanlab/bpollack/data_course/data_sets/brains/

In [ ]:
ls /pghbio/dbmi/batmanlab/bpollack/data_course/data_sets/brains/140820_YF58DB_FS

In [ ]:
# Set up the full path to the data
data_dir = '/pghbio/dbmi/batmanlab/bpollack/data_course/data_sets/brains'
patient = '140820_YF58DB_FS'
# load in a sample image using nibabel
brain_mri = nib.load(str(Path(data_dir, patient, 'norm.mgz')))
brain_seg = nib.load(str(Path(data_dir, patient, 'aseg.mgz')))

In [ ]:
print(brain_mri.header)

What does the above information tell us? Importantly, we have an image of 256x256x256 pixels with 1 value per pixel.  The physical spacing is 1mm per pixel.  There is a rotation matrix defining the transformation from image space to physical space.  For more info, follow this link: https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/MghFormat

#### *Exercise*: Make a plot of a slice of the brain (bonus: and the corresponding segmentation).

In [ ]:
# make some plots
brain_mri_img = brain_mri.get_fdata()
brain_seg_img = brain_seg.get_fdata()
# Plot the brain


The pieces are ready to go, but as it currently stands, they are not in format in which we can easily perform large-scale investigation over the entire dataset.  It would be very convenient if we could load all patients into an object that allowed quick, easy, transparent access to the data. Let's take a look at xArray.

## xArray Dataset
xArray is the multidimensional analogue to pandas.  This makes it a very attractive tool when dealing with medical imaging (especially when all the images are very uniform in shape). A Dataset has `dims`, `data_vars`, and `coords` (and user-defined `attrs` for metadata).  Let's make a simple dataset then examine it's properties.

In [ ]:
# initialize an xarray object and assign names
ds = xr.Dataset({'MRI': (['subject', 'w', 'h', 'd'],
                            np.zeros((4, 256, 256, 256), dtype=np.float32)),
                 'seg': (['subject', 'w', 'h', 'd'],
                            np.zeros((4, 256, 256, 256), dtype=np.float32)),
                },
                coords={'subject': ['140820_YF58DB_FS', '110306_QW55XU_FS', '131205_SX49HC_FS', '090706_JW54JH_FS'],
                       'w':range(256),
                       'h':range(256),
                       'd':range(256),
                       }
               )

In [ ]:
ds

In [ ]:
# fill the xarray object with the patient images
for subj in ['140820_YF58DB_FS', '110306_QW55XU_FS', '131205_SX49HC_FS', '090706_JW54JH_FS']:
    # Much like pandas, when assigning new values to a variable, it's safest to use 'loc'
    # to guarantee that we are assiging to the actual dataset and not a copy
    tmp_mri = nib.load(str(Path(data_dir, subj, 'norm.mgz'))).get_fdata()
    tmp_seg = nib.load(str(Path(data_dir, subj, 'aseg.mgz'))).get_fdata()
    ds['MRI'].loc[{'subject':subj}] = tmp_mri
    ds['seg'].loc[{'subject':subj}] = tmp_seg

In [ ]:
ds

In [ ]:
ds.sel(subject='140820_YF58DB_FS', d=100)['MRI']

In [ ]:
# Now let's plot via the xarray
fig, axes = plt.subplots(1, 2, figsize=(15,8))
axes = np.ravel(axes)
axes[0].imshow(ds.sel(subject='131205_SX49HC_FS', w=100).MRI)
axes[1].imshow(ds.sel(subject='131205_SX49HC_FS', w=100).seg, cmap=seg_cmap)

## Advanced Visualization
Let's go beyond matplotlib.  In this section, we will explore the usage of holoviews for interactive visualization.  Holoviews works natively with xArray objects, which will be to our benefit.  We'll start simple and scale up to a simple function that handles a lot of our visualization needs.

### Simple Slice Viewing with Holoviews
Before we get too fancy, let's reproduce the type of plots we've already made in matplotlib, but using holoviews.

In [ ]:
img = hv.Image(ds.sel(w=100, subject='131205_SX49HC_FS'), vdims=['MRI'], kdims=['d', 'h'])
# img.opts(invert_yaxis=True, width=500, height=500, cmap='viridis')
img.opts(width=500, height=500, cmap='viridis')

#### *Exercise*: Why do we have to flip the y axis? How can we do this within the data, before plotting?
https://dsp.stackexchange.com/questions/35925/why-do-we-use-the-top-left-corner-as-the-origin-in-image-processing

In [ ]:
# Reverse the coordinates for x, y, z.  This gives us our standard views of the brain for axial, coronal, and sagittal.
ds = ds.assign_coords(w=range(256)[::-1], h=range(256)[::-1], d=range(256)[::-1])

In [ ]:
(hv.Image(ds.sel(w=128, subject='131205_SX49HC_FS'), vdims=['MRI'], kdims=['d', 'h'], label='Sagittal')
 + hv.Image(ds.sel(h=128, subject='131205_SX49HC_FS'), vdims=['MRI'], kdims=['w', 'd'], label='Axial')
 + hv.Image(ds.sel(d=128, subject='131205_SX49HC_FS'), vdims=['MRI'], kdims=['w', 'h'], label='Coronal'))

### Multi-slice Viewing
So far, there's been no real benefit for using holoviews over standard matplotlib (in fact, holoviews is a little more cumbersome).  That's all about to change.  To maximize our benefit from holoviews, we'll make a holoviews dataset directly from our xarray object, and that will be used for our images.

In [ ]:
# Making our holoviews dataset
hv_ds = hv.Dataset(ds['MRI'])
hv_ds

In [ ]:
# Set some default options for "Image"
from holoviews import opts
opts.defaults(opts.Image(cmap='viridis', width=400, height=400, tools=['hover']))
              
hv_brain_image = hv_ds.to(hv.Image, groupby=['subject', 'w'], dynamic=True, kdims=['d', 'h'])
print(hv_brain_image)
hv_brain_image

In [ ]:
# One of the downsides of using 'dynamic=True' is that holoviews can't figure out an appropriate colormap range for the whole image so, each slice has it's own.
# Let's try to do something about that.
# Note, we can get the original xarray format by calling 'data' on our holoviews dataset
print(hv_ds.data.MRI.min())
print(hv_ds.data.MRI.max())

In [ ]:
hv_brain_image.redim.range(MRI=(0, 150))

In [ ]:
# Making our holoviews datasets
hv_ds_MRI = hv.Dataset(ds['MRI']) # MRI
hv_ds_seg = hv.Dataset(ds['seg']) # Segmentation

In [ ]:
# Make two dynamic images, plot them side by side
dyn_MRI = hv_ds_MRI.to(hv.Image, groupby=['subject', 'w'], dynamic=True, kdims=['d', 'h']).redim.range(MRI=(0,150))
dyn_seg = hv_ds_seg.to(hv.Image, groupby=['subject', 'w'], dynamic=True, kdims=['d', 'h']).opts(cmap=seg_cmap).redim.range(seg=(0,255))
# use '+' to plot both side by side
dyn_MRI+dyn_seg

Let's look at a particular segmentation value and see if we can isolate it.

In [ ]:
# See our segmentation values
np.unique(ds['seg'], return_counts=True)

In [ ]:
# Make a new segmentation dataset
# Select seg==2, make all the other values 0
hv_ds_seg_single = hv.Dataset(ds.where(ds.seg==2).fillna(0)['seg']) 

In [ ]:
dyn_seg_single = hv_ds_seg_single.to(hv.Image, groupby=['subject', 'w'],
                                     dynamic=True, kdims=['d', 'h']).opts(cmap=seg_cmap).redim.range(seg=(0,255))
# use '+' to plot both side by side
dyn_MRI+dyn_seg_single

### Alpha Blending
Overlaying images on top of one another is a very useful technique for quick verification and comparison.  There are many ways to approach this, but I prefer alpha blending, in which an image with variable transparency is overlaid on another.  Let's try to accomplish that in the holoviews framework.

In [ ]:
import panel as pn
# make a new hv dataset, but set all 0s to NaNs (NaNs are transparent)
hv_ds_seg_overlay = hv.Dataset(ds.where(ds.seg>0)['seg']) 
dyn_seg_overlay = hv_ds_seg_overlay.to(hv.Image, groupby=['subject', 'w'],
                                     dynamic=True, kdims=['d', 'h']).opts(cmap=cmap_seg).redim.range(seg=(0,255))

# Make an alpha slider
slider = pn.widgets.FloatSlider(start=0, end=1, value=0.0, name='mask')
# Plot the slider and the overlayed images using the '*' operator
pn.Column(slider, dyn_MRI.redim.range(MRI=(0,150))*dyn_seg_overlay.apply.opts(alpha=slider.param.value))